# Coref resolution

In [29]:
import pickle

# load saved model
with open('allen_coref' , 'rb') as f:
    predictor = pickle.load(f)

text = "A cat is walking. It tripped and got hurt fell on a stone. It died. Dog just walked past. It ran."

prediction = predictor.predict(document=text)  # get prediction
print("Clsuters:-")
for cluster in prediction['clusters']:
    print(cluster)  # list of clusters (the indices of spaCy tokens)
# Result: [[[0, 3], [26, 26]], [[34, 34], [50, 50]]]
print('\n\n') #Newline

coref_text = predictor.coref_resolved(text)

print('Coref resolved: ', coref_text)  # resolved text
# Result: Joseph Robinette Biden Jr. is an American politician who is the 46th and

Clsuters:-
[[0, 1], [5, 5], [15, 15]]
[[18, 18], [23, 23]]



Coref resolved:  A cat is walking. A cat tripped and got hurt fell on a stone. A cat died. Dog just walked past. Dog ran.


# SAO parsing

In [18]:
# !pip install textacy

In [19]:
from subject_verb_object_extract import findSVOs, nlp
# coref_text = "A cat is walk rapidly on a sunny day. A cat fall on a stone. A cat dead"
tokens = nlp(coref_text)
SVOs = findSVOs(tokens)
print(coref_text)
print(SVOs)

A cat is walking. A cat tripped and got hurt fell on a stone. A cat died. Dog just walked past. Dog ran.
[('A cat', 'walking'), ('A cat', 'died'), ('Dog', 'walked'), ('Dog', 'ran')]


In [20]:
from nltk.corpus import stopwords
from collections import defaultdict
from nltk.tokenize import word_tokenize
  
stop_words = set(stopwords.words('english'))
svos = defaultdict(list)

for svo in SVOs:  
    word_tokens = word_tokenize(svo[0])
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    svos["".join(filtered_sentence)].append(svo[1])
    
svos = dict(svos)
print(svos)

charcaters = list(svos.keys())
print(charcaters)

{'cat': ['walking', 'died'], 'Dog': ['walked', 'ran']}
['cat', 'Dog']


In [21]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

actions_movement = {'die' : 0, 'fall' : 0, 'hurt' : 0, 'idle' : 0, 'jump' : 1, 'run' : 1, 'slide' : 1, 'walk' : 1}
#word2vec similarity between the incoming action vs the ones we have in list of actions and then set a threshold and execute the action based on it

In [22]:
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\shour\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shour\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [23]:
import itertools
for actions in list(itertools.chain(*svos.values())):
    action = WordNetLemmatizer().lemmatize(actions,'v')
    print(action)

print(svos)


walk
die
walk
run
{'cat': ['walking', 'died'], 'Dog': ['walked', 'ran']}


In [24]:
# !pip install pygame

In [35]:
from numpy import character
import pygame
import glob
from time import sleep
 
SIZE = WIDTH, HEIGHT = 1280, 720 #the width and height of our screen
FPS = 9 #Frames per second

screen = pygame.display.set_mode(SIZE)
bg = pygame.transform.scale(pygame.image.load('pygame\sunny_day.png') , SIZE)
 
class MySprite(pygame.sprite.Sprite):
    def __init__(self, char, x, y):
        super(MySprite, self).__init__()
        
        self.images = {}
        self.actions = []
        # self.images = [pygame.image.load(img) for img in glob.glob("pygame\\cat\\*.png")]
        for action in svos[char]:
            action = WordNetLemmatizer().lemmatize(action,'v')            
            self.actions.append(action)
            print(self.actions)
            
            if action not in self.images:
                self.images[action] = [pygame.transform.scale(pygame.image.load(img) , (200,200)) for img in glob.glob("pygame\\" + char + "\\" + action + "*.png")]
            
        print(self.images)
        self.char = char
        self.index = 0
        self.action_count = 0
        self.dir = 'r'
        self.x = x
        self.y = y   

    def movement_update(self):
        if self.x == 880:
            self.dir = 'l'
        elif self.x == 0:
            self.dir = 'r'

        if self.dir == 'r':                        
            self.x += FPS
        else:
            self.image = pygame.transform.flip(self.image, True, False)
            self.x -= FPS
        screen.blit(self.image, (self.x % 880, self.y))
 
    def update(self):
        print(self.index, self.action_count, len(self.actions), len(self.images[self.actions[self.action_count]]))
        if self.index >= len(self.images[self.actions[self.action_count]]):
            if self.action_count >= len(self.actions) - 1:
                return 1
            self.index = 0
            self.action_count += 1
        self.image = self.images[self.actions[self.action_count]][self.index]
        self.index += 1

        if actions_movement[self.actions[self.action_count]]:
            self.movement_update()
        else:
            screen.blit(self.image, (self.x, self.y))
        
        return 0
        
 
def main():
    pygame.init()
    pygame.display.set_caption("Trace")
    chars = []
    for i in range(len(charcaters)):
        chars.append(MySprite(charcaters[i], (200 * i), 350))
    # my_group = pygame.sprite.Group(my_sprite)
    clock = pygame.time.Clock()
    
    loop = 1
    count = 0
    while count < len(charcaters):   
        
        screen.fill((0,0,0))        
        screen.blit(bg, (0, 0))

        for i in chars:
            count += i.update()        
        pygame.display.update()
        clock.tick(FPS)
    pygame.quit()
 
if __name__ == '__main__':
    main()

['walk']
['walk', 'die']
{'walk': [<Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>], 'die': [<Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>]}
['walk']
['walk', 'run']
{'walk': [<Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>], 'run': [<Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <

# NER

In [26]:
import spacy 
import spacy.cli
# Download following once
#spacy.cli.download("en_core_web_sm")

In [27]:
from spacy import displacy

nlp = spacy.load('en_core_web_sm')

In [28]:
text = "Victoria was a major in CS. Victoria studies at MIT."
doc = nlp(text)

ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print(ents)
# now we use displaycy function on doc2
displacy.render(doc, style='ent', jupyter=True)

[('CS', 24, 26, 'ORG'), ('MIT', 48, 51, 'ORG')]
